<a href="https://colab.research.google.com/github/bartek717/projectx/blob/main/FLAN_T5_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset
import torch
# here you can change the model if you want. Looks like the available options are (flan-t5-) small, base, large, xl, xxl. Probably easiest to use one of the smaller models for our use case.
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.6 MB/s eta 0:00:00


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
# subpubmed or whatever preprocessing
!git clone https://github.com/vgupta123/sumpubmed.git
%cd sumpubmed/

Cloning into 'sumpubmed'...
remote: Enumerating objects: 130808, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 130808 (delta 22), reused 17 (delta 6), pack-reused 130764
Receiving objects: 100% (130808/130808), 346.82 MiB | 11.79 MiB/s, done.
Resolving deltas: 100% (38368/38368), done.
Updating files: 100% (130765/130765), done.
/content/sumpubmed


In [ ]:
import os

def process_data(text_dir, summary_dir):
    texts = []
    summaries = []

    # Assuming the file names in both directories match
    for filename in os.listdir(text_dir):
        if filename.endswith(".txt"):
            # Read the full text
            with open(os.path.join(text_dir, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())

            # Read the corresponding summary
            summary_file = "abstract_"+(filename.split("_")[-1])
            with open(os.path.join(summary_dir, summary_file), 'r', encoding='utf-8') as file:
                summaries.append(file.read())

    return texts, summaries


In [ ]:
texts, summaries = process_data('text/', 'abstract/')


In [ ]:
def tokenize_function(examples):
    # Tokenize the inputs and labels
    tokenized_inputs = tokenizer(examples["texts"], padding=True, truncation=True, max_length=1024)
    with tokenizer.as_target_tokenizer():
        tokenized_labels = tokenizer(examples["summaries"], padding=True, truncation=True, max_length=256)

    tokenized_inputs["labels"] = tokenized_labels["input_ids"]
    return tokenized_inputs

In [ ]:
dataset = Dataset.from_dict({"texts": texts, "summaries": summaries})
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/32689 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
#make an 80/20 split
total_examples = len(tokenized_datasets)
train_size = int(0.8 * total_examples)
eval_size = total_examples - train_size

# Split the dataset
train_dataset = tokenized_datasets.shuffle(seed=42).select(range(train_size))
eval_dataset = tokenized_datasets.shuffle(seed=42).select(range(train_size, total_examples))


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Fine-tuning the model
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.579400,2.366883
2,2.523100,2.331602
3,2.487600,2.323345


TrainOutput(global_step=39228, training_loss=2.557207064173012, metrics={'train_runtime': 5987.8599, 'train_samples_per_second': 13.102, 'train_steps_per_second': 6.551, 'total_flos': 2.9167335531085824e+16, 'train_loss': 2.557207064173012, 'epoch': 3.0})

In [ ]:
evaluation_results = trainer.evaluate()

In [ ]:
# this is where we add our hallucination eval - tbd

In [ ]:
# Example: Generating predictions for a single input
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

input_text = """BACKGROUND
trichophyton rubrum is a cosmopolitan dermatophyte that colonizes human skin and nails and is the most prevalent cause of human dermatophytoses  <cit> . during the initial stages of the infection, dermatophytes induce the expression of adhesins and unspecific proteases and keratinases that have optimum activity at acidic ph values  <cit> , which is probably because the human skin has an acidic ph value  <cit> . the secretion of these proteases, which have been identified as an important step in fungal pathogenicity and virulence  <cit> , act on keratinous and nonkeratinous substrates to release peptides that are further hydrolyzed to amino acids by putative peptidases. the metabolism of some amino acids shifts the extracellular ph from acidic to alkaline values at which most known keratinolytic proteases have optimal enzymatic activity  <cit> . t. rubrum also responds to the environmental ph by altering its gene expression profile  <cit> .

molecular studies have been performed with human pathogens such as candida albicans, histoplasma capsulatum, and paracoccidioides brasiliensis, and the results thus obtained have helped to determine the fungal transcriptional profile and characterize the genes involved in host-pathogen interactions and environmental stress responses  <cit> . previously, a collection of t. rubrum expressed sequence tags  was obtained from distinct developmental phases  <cit> . however, determining the transcriptional profiles in response to different cell stimuli is necessary for extending our understanding of diverse cellular events, and the results from such studies may reveal new signal transduction networks and the activation of specific metabolic pathways. functional analysis of the genes involved in these molecular events will help in evaluating their roles as putative cellular targets in the development of new antifungal agents.

our study aimed to extend the t. rubrum genomic database by adding expressed gene resources that cover different aspects of cellular metabolism. moreover, the data can help to generate useful information to screen valuable genes for functional and postgenomic analyses. the est collection described here revealed the metabolic adaptations of the human pathogen t. rubrum to changes in the ambient ph and carbon sources and also provided information on the adaptive responses to several cytotoxic drugs.

RESULTS
the est collection described here was obtained from a cdna library and nine independent suppression subtractive hybridization  libraries. a total of  <dig> high-quality sequences with an average sequence length of  <dig> nucleotides were generated. these ests were assembled in  <dig> contigs and  <dig> singletons, resulting in  <dig> unique sequences with a redundancy of  <dig> % . the majority of the contigs assembled ests from a maximum of four libraries, suggesting that these genes are expressed under environmental stress or specific growth conditions. the search results and genbank submission numbers for each est are shown in additional file  <dig>  analysis of these  <dig> unigenes revealed  <dig> sequences that had no similarity to the sequences in the genbank dbest, which contains  <dig> t. rubrum sequences. of the  <dig> sequences,  <dig> had no similarities to the sequences in the nonredundant database . additional analysis revealed that of the  <dig> sequences,  <dig> were present in the tred database  <cit> . thus,  <dig> novel genes were identified, representing a marked increase in the number of expressed genes identified in the dermatophyte t. rubrum. these genes and the corresponding libraries in which they were identified are highlighted in additional file  <dig>

 unigenes with similarity to the sequences in the nonredundant ncbi database  using blastx.

 unigenes with no similarity to the sequences in the dbest-ncbi database  using blastn-organism: trichophyton rubrum .

 t. rubrum protein sequences identified in this database were excluded from this analysis.

the  <dig> unigenes identified in this study were functionally classified based on the munich information center for protein sequences  categories. the classification led to the identification of putative proteins involved in transcriptional regulation, cellular defense and stress, protein degradation, signaling, transport, and secretion, among other functions . however, many of these unigenes  were not classified by the mips annotation . growth of t. rubrum in a keratinocyte serum-free medium  revealed  <dig> novel genes  in comparison to the t. rubrum sequences deposited in public databases, which include an est collection that was previously generated during the growth of t. rubrum in sabouraud liquid medium  <cit> . this suggests that the expression of these  <dig> novel genes is nutrient-dependent. functional grouping of these genes, which were identified on the basis of their ests, revealed their possible involvement in various cellular processes such as basic metabolism, conidial germination, and hyphal growth, among other functions .

challenging t. rubrum with cytotoxic drugs
numerous signal-transduction pathways are used by fungi to sense and overcome the toxic effects of antifungal drugs  <cit> . our aim in this study was to identify metabolic events that occur during the initial stages of drug exposure; therefore, we created an est collection by challenging the dermatophyte t. rubrum with cytotoxic drugs, including most of the antifungals used in medical practice. these drugs, which belong to the azole and allylamine/thiocarbamate classes, were fluconazole , imazalil , itraconazole , ketoconazole , tioconazole , and terbinafine . all of these compounds inhibit the biosynthesis of ergosterol. t. rubrum was also challenged with the following cytotoxic drugs: amphotericin b , griseofulvin , benomyl , undecanoic acid , cycloheximide , chloramphenicol , acriflavin , ethidium bromide , and 4-nitroquinoline 1-oxide   <cit> . approximately  <dig> unigenes were identified in these experiments and only  <dig> of these were exclusive to drug challenge . drug exposure induced the transcription of several multidrug resistance genes, as previously reported in studies in which t. rubrum was exposed to sub-inhibitory levels of ktc, amb, or other drugs  <cit> . one of these genes  encodes a putative multidrug resistance protein  that accumulates in the mycelia when the organism is independently exposed to various cytotoxic agents. overexpression of this gene has been previously reported in the myceliaof t. rubrum exposed to the antimycotic agents acr, grs, itra, or flc  <cit> . disruption of this gene increased the susceptibility of the mutant strain to trb in comparison with the control, suggesting that this transporter modulates t. rubrum drug susceptibility  <cit> .

some of the ests that were overexpressed in the mycelia of t. rubrum upon exposure to cytotoxic drugs showed similarities to genes encoding the pol protein , carboxylic ester hydrolase , dna mismatch repair protein , and cooper resistance-associated p-type atpase protein , among others. these results had been previously validated by northern blot analyses in mycelia of t. rubrum grown in the presence of trb or grs  <cit> . upregulation of ests similar to the pol gene of the cgret retrotransposon element from glomerella cingulata  suggests that t. rubrum evinces an adaptive response to environmental stress. interestingly, overexpression of this gene was also observed in mycelia of t. rubrum grown in keratin as the carbon source , which suggests the involvement of this retrotransposon in nonspecific responses, leading to stress adaptation.

overexpression of an est encoding salicylate 1-monooxigenase, a naphthalene-degrading enzyme  , was exclusive to t. rubrum that had been challenged with cytotoxic drugs, including trb . a possible mechanism of resistance to trb was evidenced in the model fungus aspergillus nidulans and involved the overexpression of the salicylate 1-monooxigenase gene sala, probably due to a multicopy effect  <cit> . moreover, plasmids carrying the sala gene of a. nidulans were able to transform a t. rubrum strain from trb-sensitive to trb-resistant, suggesting that a similar resistance mechanism could help t. rubrum to overcome the inhibitory effect of trb, which has a naphthalene nucleus present in its molecular structure .

ph and carbon source signaling
among the most important virulence factors identified in dermatophytes are proteases that have optimal activity at acidic ph and are secreted during the initial stages of fungal infection  <cit> . the hydrolysis of skin proteins releases amino acids such as glycine, the metabolism of which shifts the extracellular ph from acidic to alkaline values  <cit> . this effect is required for the growth and maintenance of the dermatophyte in the host  <cit> . therefore, identification of t. rubrum genes that are differentially expressed in response to shifts in the ambient ph provides useful information on ph sensing during host infection. when the media was supplemented with glucose as the carbon source, we identified  <dig> genes that were overexpressed at ph  <dig>  and  <dig> genes that were overexpressed in response to alkaline ph conditions . functional grouping of these ests led to the identification of genes involved in various cellular processes, such as membrane remodeling, cellular transport, iron uptake, defense, metabolism, signal transduction, and virulence.

interestingly, the transcription of the gene encoding an acetamidase  was stimulated in an acidic milieu . this enzyme hydrolyses acetamide, releasing acetate and ammonia. acetate is metabolized to acetyl-coa by acetyl-coa synthase, with the concomitant secretion of ammonia, which raises the extracellular ph to alkaline values  <cit> . the metabolism of amino acids that generate cytoplasmic acetyl-coa shifts the extracellular ph from acidic to alkaline values  <cit> , an effectobserved in in vitro cultures of t. rubrum  <cit> . the metalloenzyme urease  catalyzes the hydrolysis of urea to ammonia during the parasitic cycle of coccidioides posadasii and also creates an alkaline microenvironment at the infection site. ammonia secretion contributes to host tissue damage, thereby enhancing the virulence of this human respiratory pathogen  <cit>  .

the overexpression of the ests from ssh libraries was confirmed by reverse northern hybridization and/or northern blot.

overexpression of some genes under acidic ph conditions was confirmed by northern blot analysis . one of these encodes a protein carrying the fyve zinc finger domain . fyve domains are found in several eukaryotic nonnuclear proteins that are involved in many cellular functions, including cytoskeletal regulation, signal transduction, and vesicle transport  <cit> . most of the proteins that carry the fyve domain function in the recruitment of cytosolic proteins by binding to phosphatidylinositol 3-phosphate, which is mainly found in the endosome and functions as a regulator of endocytic membrane trafficking  <cit> . interestingly, the anchoring of fyve proteins to phosphatidylinositol 3-phosphate-enriched membranes is strongly ph-dependent and is enhanced by an acidic cytosolic environment  <cit> . a relevant gene that is overexpressed at alkaline ph values encodes an iron-sulfur cluster protein , a cofactor for several proteins involved in electron transfer in redox and nonredox catalysis, in gene regulation, and as sensors of oxygen and iron  <cit> . some genes involved in the acquisition of iron by c. albicans are also overexpressed at ph  <dig> , suggesting that alkaline ph induces iron starvation  <cit> . thus, genes overexpressed at either acidic or alkaline ph values are probably involved in the initial stages of dermatophyte infection and maintenance in the host tissue, respectively.

identification of the ests involved in keratin metabolism may also help in determining the genes necessary for installation and maintenance of the pathogen in the host. we identified  <dig> keratin-enriched transcripts, and  <dig> ests which were involved in glucose metabolism . it was previously observed that the ph of the medium remained at a value of approximately  <dig>  during mycelial growth when glucose was the carbon source. in contrast, during growth on keratin or a mixture of glycine and glycerol, the extracellular ph shifted from acidic to alkaline, reaching ph values from  <dig>  to  <dig>   <cit> . growth on keratin at alkaline ph values revealed the overexpression of several proteases and membrane transporter protein genes  such as subtilisin protease sub  <dig> , metalloprotease mep <dig> , mfs oligopeptide transporter , mdr protein , cu2+-atpase , v-type atpase, subunit b , and an aminoacid permease   <cit> . most of these genes were not overexpressed when the initial culture ph was adjusted to  <dig>  and glucose was used as the carbon source  . this suggests that a combination of an ambient ph shift and keratin as the carbon source is necessary to induce the expression of these genes. interestingly, the gene encoding nima interactive protein  was overexpressed in keratin cultures, in response to cytotoxic drugs, and after mycelial exposure for  <dig> min at ph  <dig> , suggesting that this gene may be involved in unspecific stress responses. overexpression of the nima interactive protein gene in mycelia of t. rubrum exposed to acid ph  or grown in keratin as the only carbon source  was confirmed by northern blot analysis. in fact, this protein is a member of the nima family of kinases and is expressed in response to unspecific cellular stresses  <cit> . furthermore, the hsp <dig> gene  and a transcript with no significant similarity  were confirmed to be overexpressed when keratin was used as the carbon source . the hsp <dig> gene of saccharomyces cerevisiae is strongly induced when the fungus is exposed to various stresses, including heat shock and glucose starvation  <cit> . similar to many other heat shock proteins, hsp <dig> increases cellular tolerance to stress.

genes that contribute to virulence
the ests shown in table  <dig> reveal t. rubrum genes that encode putative proteins similar to the virulence factors identified in other fungi. three of the five glyoxylate cycle enzymes were identified in our est database, i.e., isocitrate lyase and malate synthase, which are key enzymes of this cycle, together with citrate synthase. the glyoxylate cycle is required for the full virulence of c. albicans  <cit> , mycobacterium tuberculosis  <cit> , and p. brasiliensis  <cit> . moreover, nutritional stress conditions in vitro also require upregulation of the glyoxylate cycle enzymes in p. brasiliensis  <cit> .

secreted enzymes such as phospholipases, peptidases, and proteases are crucial for dermatophyte virulence since these pathogens infect the stratum corneum, nails, or hair  <cit> . during infection, t. rubrum carboxypeptidases may contribute to fungal virulence by cooperating with endoproteases and aminopeptidases to degrade compact keratinized tissues into short peptides and amino acids that can be assimilated  <cit>  .

various membrane transporters are virulence factors that are commonly involved in bacterial and fungal pathogenesis and ensure successful colonization of the host. for example, transmembrane proteins involved in the transport of metallic ions appear to play an important role in microbial pathogenesis  <cit>  as demonstrated in the cu2+-atpase mutants of listeria monocytogenes  <cit>  and criptococcus neoformans  <cit>  that show reduced virulence. in the latter case, the Δvph <dig> mutant did not display laccase activity, which is an essential virulence factor of this pathogen  <cit> . moreover, an atp-binding cassette  transporter listed in table  <dig> is overexpressed in mycelia cultured in keratin, suggesting its involvement in t. rubrum pathogenicity. in addition, the strain carrying a disrupted version of this mdr gene  showed low infectious capability characterized by reduced growth of t. rubrum on human nails  <cit> .
"""
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
inputs = inputs.to(device)

output = model.generate(**inputs)
predicted_summary = tokenizer.decode(output[0], skip_special_tokens=True)

print("Predicted Summary:", predicted_summary)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicted Summary: BACKGROUND trichophyton rubrum is a cosmopolita


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model_output_dir = '/content/drive/My Drive/t5fine'
tokenizer_output_dir = '/content/drive/My Drive/t5fine'
model.save_pretrained(model_output_dir)
tokenizer.save_pretrained(tokenizer_output_dir)

('/content/drive/My Drive/t5fine/tokenizer_config.json',
 '/content/drive/My Drive/t5fine/special_tokens_map.json',
 '/content/drive/My Drive/t5fine/tokenizer.json')